In [14]:
# IMPORT DEPENDANCIES

import pandas as pd
import json
import requests
# !pip install rebrick
import rebrick

https://github.com/xxao/rebrick

In [15]:
# INIT REBRICK API MODULE 
api_key = open('creds.txt','r').read()
rebrick.init(api_key)

In [6]:
# IMPORT MY SET INVENTORY
sets = pd.read_csv('rebrickable_sets_taylorsyde.csv').rename({'Set Number':'set_num'})

# create col for set id needed to API call
sets['api_id'] = sets['Set Number'].str.replace('-1','')

In [7]:
# REQUEST ALL CURRENT THEMES FROM API & STORE
themes_response = json.loads((rebrick.lego.get_themes()).read())
themes = pd.DataFrame(themes_response['results'])[['id','name']].rename(columns = {'name':'theme_name', 'id':'theme_id'})

themes.to_csv('lego_themes.csv', index=False)

In [8]:
# REQUEST SET INFO FROM API
set_responses = []

for s in sets['api_id']: 
    # get set overview info
    set_response = rebrick.lego.get_set(s)
    set_response_dict = json.loads(set_response.read())
    set_data = pd.DataFrame(set_response_dict, index=[0])
    set_responses.append(set_data)

# create set df
df_sets = pd.concat(set_responses)

#### Get Parts and Prep

In [35]:
# PREP PARTS DATA

# select target columns from nested dictonary
df_parts[['part_number','part_name', 'part_category', 
          'part_image_url','part_url','color_id', 'color_name']] = None

# extract info from embedded dictionarys 
for i, row in df_parts.iterrows():
    df_parts.at[i, 'part_number'] = row['part'].get('part_num')
    df_parts.at[i, 'part_name'] = row['part'].get('name')
    df_parts.at[i, 'part_category'] = row['part'].get('part_cat_id')
    df_parts.at[i, 'part_image_url'] = row['part'].get('part_img_url')
    df_parts.at[i, 'part_url'] = row['part'].get('part_url')
    
    df_parts.at[i, 'color_id'] = row['color'].get('id')
    df_parts.at[i, 'color_name'] = row['color'].get('name')

In [32]:
part_responses = []

for s in sets['Set Number']: 
    print(f'Getting parts for set {s}')
    next_page = True
    page_number = 1
    parts_list = []

    while next_page: 
        part_response = json.loads((rebrick.lego.get_set_elements(s, page=page_number)).read())
        parts_list.extend(part_response['results']) # add the page to the parts list
        next_page = part_response['next'] is not None # checks for next page value
        page_number += 1 # increase the page count

    # convert to DataFrame & append to full parts list
    part_data = pd.DataFrame(parts_list)
    part_responses.append(part_data)
    
# create one master df of parts
df_parts = pd.concat(part_responses).reset_index(drop=True)

Getting parts for set 10266-1
Getting parts for set 10280-1
Getting parts for set 21328-1
Getting parts for set 76989-1
Getting parts for set 10311-1
Getting parts for set 10313-1
Getting parts for set 10314-1
Getting parts for set 40587-1
Getting parts for set 77013-1
Getting parts for set 77015-1
Getting parts for set 21342-1


In [36]:
df_parts.sample()

,id,inv_part_id,part,color,set_num,quantity,is_spare,element_id,num_sets,part_number,part_name,part_category,part_image_url,part_url,color_id,color_name
2092,16729832,16729832,"{'part_num': '90981', 'name': 'Insect Accessor...","{'id': 15, 'name': 'White', 'rgb': 'FFFFFF', '...",77015-1,1,False,6006221,57,90981,"Insect Accessory, Spider Web, Hanging",28,https://cdn.rebrickable.com/media/parts/elemen...,https://rebrickable.com/parts/90981/insect-acc...,15,White


#### Merge Into Final Output

In [38]:
# SELECT TARGET COLUMNS 
parts_clean = df_parts[['set_num', 'part_number', 'part_name', 'part_category'
                       ,'part_image_url', 'part_url', 'color_id', 'color_name'
                       , 'quantity', 'is_spare']]

sets_clean  = df_sets[['set_num', 'name', 'year', 'theme_id', 'num_parts' 
                      ,'set_img_url','set_url']]

In [39]:
parts_clean.quantity.sum()

10621

In [40]:
master_inventory = sets_clean.merge(parts_clean, how = 'outer', on = 'set_num')

In [41]:
master_inventory.quantity.sum()

10621

In [42]:
master_inventory.to_csv('lego_inventory.csv', index=False)